# Desarrollo del Proyecto 1 - Recomendación de películas
Hecho por: Alejandra Salas

### Modelo de Predicción_Sistema de recomendación

## 1. Extraer los datos

In [1]:
# Se importan las librerías a usar.
import numpy as np
import pandas as pd

In [2]:
datos=pd.read_csv('datos_main.csv')

In [3]:
# Importamos las bibliotecas a utilizar en el modelo
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [5]:
# Convertimos los datos en tipo str.
datos['title'] = datos['title'].fillna('').astype('str')
datos['genres'] = datos['genres'].fillna('').astype('str')
datos['companies'] = datos['companies'].fillna('')
datos['companies'] = datos['companies'].fillna('').astype('str')

In [9]:
# Creamos una nueva columna combinando las características de interés
datos['caracteristicas_comunes'] = datos['title'] + ' ' + datos['genres'] + ' ' + datos['companies']

# Convertimos todos los textos a minusculas para evitar duplicados
datos['caracteristicas_comunes'] = datos['caracteristicas_comunes'].str.lower()

#   Creamos una matriz de conteo usando CountVectorizer que convierte los textos en una matriz de frecuencias de palabras.
contar = CountVectorizer(stop_words='english', max_features=5000)
count_matrix = contar.fit_transform(datos['caracteristicas_comunes'])

# Creamos un modelo para encontrar los vecinos mas cercanos en un espacio de caracteristicas
modelo_k = NearestNeighbors(metric='cosine', algorithm='brute')
modelo_k.fit(count_matrix)

# Creamos un indice de titulos de peliculas y eliminamos los duplicados
indices = pd.Series(datos.index, index=datos['title']).drop_duplicates()

In [35]:
# Definimos la función.
def recomendacion(titulo):
    if titulo not in datos['title'].values:
        return 'La pelicula no se encuentra en el conjunto de la base de datos.'
    else:
        index = indices[titulo]

        distances, indices_knn = modelo_k.kneighbors(count_matrix[index], n_neighbors=6)  

        movie_indices = indices_knn[0][1:]  

        return {'lista recomendada': datos['title'].iloc[movie_indices].tolist()}


In [36]:
print(recomendacion("Toy Story"))

{'lista recomendada': ['Toy Story 2', 'Toy Story 3', 'Toy Story 3', 'Toy Story of Terror!', 'Toy Story That Time Forgot']}
